## 1. Persiapan

In [ ]:
# import library yang dibutuhkan
import sqlite3
import pandas as pd

In [ ]:
from google.colab import drive

In [ ]:
# cek versi dari sqlite kita
sqlite3.sqlite_version

'3.37.2'

Ada 2 cara untuk membuat koneksi ke database

In [ ]:
conn = sqlite3.connect('northwind.db')

In [ ]:
# membuat koneksi ke database via path
conn = sqlite3.connect('/content/northwind.db')

## 2. Mengakses Database

In [ ]:
# buat variabel cursor dari koneksi ke database kita
cursor = conn.cursor()

In [ ]:
# tuliskan query yang akan kita jalankan
# query di bawah akan menampilkan tabel - tabel yang ada di database kita
query = """
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
"""

In [ ]:
# menjalankan query dan menampilkan hasil
cursor.execute(query)
print(cursor.fetchall())

[('Suppliers',), ('Shippers',), ('Employees',), ('Customers',), ('Orders',), ('Categories',), ('Products',), ('OrderDetails',)]


# 6. Study Case

In [ ]:
# Tampilkan 10 teratas kolom ProductName, UnitPrice, Quantity, Discount, dan ShipCountry dan urutkan berdasarkan Quantity termahal


query1 = """
SELECT
  p.ProductName,
  p.UnitPrice,
  od.Quantity,
  od.Discount,
  o.ShipCountry
FROM Products p
JOIN orderdetails od ON p.ProductID = od.ProductID
JOIN orders o ON od.OrderID = o.OrderID
ORDER BY 2 DESC
LIMIT 10
"""
pd.read_sql_query(query1,conn)

,ProductName,UnitPrice,Quantity,Discount,ShipCountry
0,Côte de Blaye,263.5,20,0.05,USA
1,Côte de Blaye,263.5,20,0.05,Austria
2,Côte de Blaye,263.5,50,0.20,Austria
3,Côte de Blaye,263.5,10,0.00,France
4,Côte de Blaye,263.5,40,0.25,Brazil
5,Côte de Blaye,263.5,50,0.00,Denmark
6,Côte de Blaye,263.5,49,0.20,Canada
7,Côte de Blaye,263.5,30,0.00,USA
8,Côte de Blaye,263.5,15,0.00,Mexico
9,Côte de Blaye,263.5,30,0.00,Germany


In [ ]:
# Tampilkan ProductName, CategoryName, QuantityPerUnit, UnitPrice, dan UnitsInStock yang memiliki produk roti (breads) dengan stock lebih dari 40


query2 = """
SELECT
  p.ProductName,
  c.CategoryName,
  p.QuantityPerUnit,
  p.UnitPrice,
  p.UnitsInStock
FROM Products p
JOIN Categories c on p.CategoryID = c.CategoryID
WHERE c.Description LIKE '%Breads%' AND p.UnitsInStock > 40


"""
pd.read_sql_query(query2,conn)

,ProductName,CategoryName,QuantityPerUnit,UnitPrice,UnitsInStock
0,Gustaf's Knäckebröd,Grains/Cereals,24 - 500 g pkgs.,21.00,104
1,Tunnbröd,Grains/Cereals,12 - 250 g pkgs.,9.00,61
2,NuNuCa Nuß-Nougat-Creme,Confections,20 - 450 g glasses,14.00,76
3,Schoggi Schokolade,Confections,100 - 100 g pieces,43.90,49
4,Valkoinen suklaa,Confections,12 - 100 g bars,16.25,65


In [ ]:
# Hitung rata-rata diskon (Average Discount) dan total pendapatan setelah diskon untuk setiap kategori produk. Hanya tampilkan kategori dengan rata-rata diskon lebih dari 5%.
# Tampilkan kolom CategoryName, AvgDiscount, dan TotalRevenue.

query3 = """
SELECT
  c.CategoryName,
  AVG( o.Discount ) AvgDiscount ,
  SUM(o.UnitPrice * (1 - o.Discount) * o.Quantity) TotalRevenue
FROM orderdetails o
JOIN Products p ON o.ProductID = p.ProductID
JOIN Categories c ON p.CategoryID = c.CategoryID
GROUP BY 1
HAVING AVG( o.Discount) > 0.05



"""
pd.read_sql_query(query3,conn)

,CategoryName,AvgDiscount,TotalRevenue
0,Beverages,0.061881,268001.3800
1,Condiments,0.052639,106047.0850
2,Confections,0.056946,167610.8450
3,Dairy Products,0.053443,234507.2850
4,Meat/Poultry,0.064451,163022.3595
5,Seafood,0.060242,131261.7375


In [ ]:
# Tampilkan data CompanyName dan City serta hitung jumlah ordernya untuk kota Tsawassen dan Madrid

query4 = """
SELECT
  c.CompanyName,
  c.City,
  COUNT( o.OrderID) TotalOrder
FROM customers c
JOIN orders o ON c.CustomerID = o.CustomerID
GROUP BY 1
HAVING City="Tsawassen" OR City="Madrid"




"""
pd.read_sql_query(query4,conn)

,CompanyName,City,TotalOrder
0,Bottom-Dollar Markets,Tsawassen,14
1,Bólido Comidas preparadas,Madrid,3
2,Romero y tomillo,Madrid,5


In [ ]:
# Tampilkan nama pelanggan (CustomerName) yang melakukan pemesanan lebih dari sekali, bersama dengan total jumlah order mereka. Urutkan dari jumlah order terbanyak.
# Tampilkan kolom CustomerName, TotalOrders




query5 = """
SELECT
  c.ContactName,
  COUNT(o.OrderID) TotalOrders
FROM Customers c
JOIN orders o ON c.customerID=o.CustomerID
GROUP BY 1
HAVING TotalOrders > 1
ORDER BY 2 DESC



"""
pd.read_sql_query(query5,conn)

,ContactName,TotalOrders
0,Jose Pavarotti,31
1,Roland Mendel,30
2,Horst Kloss,28
3,Patricia McKenna,19
4,Maria Larsson,19
...,...,...
83,Helvetius Nagy,3
84,Elizabeth Brown,3
85,Carine Schmitt,3
86,Manuel Pereira,2
